In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import bokeh.io
import bokeh
import iqplot

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv('cns186_data.csv')

# category / num
cats = []
nums = []
hf_cat = []
confidence = []
for im, init, ans, pt in zip(df['Image'], df['Inits'], df['Answer'], df['Part']):
    im = im.split('/')
    num = im[-1].split('.')[0]
    im = im[-2]
    cats.append(im)
    nums.append(num)
    hf = df.loc[((df['Image']==im)&(df['Inits']==init))]
    hf = df.loc[((df['Freq']=='h')&(df['Inits']==init))]
    hf = hf['Image'].values[0].split('/')[-2]
    hf_cat.append(hf)
    if pt == 2:
        confidence.append(0)
    else:
        confidence.append(np.abs(ans-3.5)+.5)
df['Category'] = cats
df['Image Number'] = nums
df['HF Cat'] = hf_cat
df['Confidence'] = confidence

# correct 
truth = []
correct = []
conf_corr = []
for subj in df['Inits'].unique():
    temp = df.loc[df['Inits']==subj]
    temp_pt2 = temp.loc[temp['Part']==2]
    temp_pt4 = temp.loc[temp['Part']==4]
    pt2_imgs = temp_pt2['Image'].values
    for i in range(len(pt2_imgs)):
        truth.append(-1)
        correct.append(-1)
        conf_corr.append(0)
    for img, ans in zip(temp_pt4['Image'].values, temp_pt4['Answer'].values):
        if img in pt2_imgs:
            truth.append(1)
            if ans >= 4:
                correct.append(1)
                conf_corr.append(1)
            else:
                correct.append(0)
                conf_corr.append(-1)
        else:
            truth.append(0)
            if ans <= 3:
                correct.append(1)
                conf_corr.append(1)
            else:
                correct.append(0)
                conf_corr.append(-1)
                
df['Was in first set?'] = truth
df['Was correct?'] = correct
df['Conf + Corr'] = df['Confidence'] * conf_corr
df = df.drop(columns=['Image'])
df.head()

,Inits,Part,Answer,Freq,Category,Image Number,HF Cat,Confidence,Was in first set?,Was correct?,Conf + Corr
0,MM,2,1,h,space,18,space,0.0,-1,-1,0.0
1,MM,2,0,l,buildings,18,space,0.0,-1,-1,0.0
2,MM,2,0,l,food,44,space,0.0,-1,-1,0.0
3,MM,2,1,l,animals,43,space,0.0,-1,-1,0.0
4,MM,2,0,l,buildings,39,space,0.0,-1,-1,0.0


In [3]:
df4 = df.loc[df['Part']==4]
df4.head()

,Inits,Part,Answer,Freq,Category,Image Number,HF Cat,Confidence,Was in first set?,Was correct?,Conf + Corr
51,MM,4,2,h,space,3,space,2.0,0,1,2.0
52,MM,4,1,l,animals,5,space,3.0,0,1,3.0
53,MM,4,6,l,food,9,space,3.0,1,1,3.0
54,MM,4,1,h,space,28,space,3.0,1,0,-3.0
55,MM,4,1,l,animals,20,space,3.0,0,1,3.0


In [55]:
frac_correct_lf = []
frac_correct_hf = []
for subj in df4['Inits'].unique():
    temp = df4.loc[((df4['Inits']==subj)&(df4['Freq']=='l'))]
    pc = sum(temp['Was correct?'].values)
    t = len(temp['Was correct?'].values)
    frac_correct_lf.append(pc/t)
    temp = df4.loc[((df4['Inits']==subj)&(df4['Freq']=='h'))]
    pc = sum(temp['Was correct?'].values)
    t = len(temp['Was correct?'].values)
    frac_correct_hf.append(pc/t)

p = np.ndarray.flatten(np.array([frac_correct_lf,frac_correct_hf]))
f = [*[8]*len(frac_correct_lf), *[20]*len(frac_correct_hf)]
i = np.arange(0, len(p))

summary_df = pd.DataFrame(np.array(np.array([p, f, i]).T), columns=['frac_correct', 'freq', 'id'])

f = iqplot.stripbox(data=summary_df, q='frac_correct', cats='freq', 
                    title='Fraction Correct for Low and High Frequency Categories')
bokeh.plotting.show(f)

In [5]:
# t test
st.ttest_ind(frac_correct_lf, frac_correct_hf, equal_var=False)

Ttest_indResult(statistic=2.364138666306591, pvalue=0.025353776660493984)

In [8]:
inits = df['Inits'].unique()
hf_cat = []
performance = []
hf_perf = []
lf_perf = []
for init in inits:
    hf_cat.append(df.loc[df['Inits']==init]['HF Cat'].values[0])
    p = df.loc[((df['Inits']==init)&(df['Part']==4))]['Was correct?'].values
    performance.append(sum(p)/len(p))
    hp = df.loc[((df['Inits']==init)&(df['Part']==4)&(df['Freq']=='h'))]['Was correct?'].values
    hf_perf.append(sum(hp)/len(hp))
    lp = df.loc[((df['Inits']==init)&(df['Part']==4)&(df['Freq']=='l'))]['Was correct?'].values
    lf_perf.append(sum(lp)/len(lp))
    

participant_df = pd.DataFrame()
participant_df['Inits'] = inits
participant_df['HF Cat'] = hf_cat
participant_df['Overall Performance'] = performance
participant_df['HF Performance'] = hf_perf
participant_df['LF Performance'] = lf_perf

participant_df.head()

,Inits,HF Cat,Overall Performance,HF Performance,LF Performance
0,MM,space,0.921569,0.809524,1.000000
1,SA,people,0.745098,0.736842,0.750000
2,CZ,landscapes,0.862745,0.750000,0.935484
3,RB,phones,0.843137,0.684211,0.937500
4,MH,buildings,0.862745,0.842105,0.875000


In [19]:
perf_df = pd.melt(participant_df, id_vars=['Inits', 'HF Cat'], 
                 value_vars=['HF Performance', 'LF Performance'], 
                 var_name='Performance Type', value_name='Performance')

In [20]:
p = iqplot.strip(
    perf_df,
    q="Performance",
    cats=["Performance Type"],
    q_axis="y",
    frame_width=500,
    color_column="HF Cat",
    parcoord_column="Inits",
    title='Performance for High vs. Low Frequency Categories'
)

bokeh.io.show(p)

In [11]:
st.ttest_rel(lf_perf, hf_perf)

Ttest_relResult(statistic=2.4230297852608067, pvalue=0.02500458176086703)

In [21]:
f = iqplot.strip(data=perf_df, q='Performance', cats=['Performance Type', 'HF Cat'], 
                 marker="dash", title='High and Low Frequency Performance by HF Category')
bokeh.plotting.show(f)

In [54]:
# t tests
cats = []
pts = []
ps = []
for cat in perf_df['HF Cat'].unique():
    for pt in perf_df['Performance Type'].unique():
        p = perf_df.loc[((perf_df['HF Cat']==cat)&(perf_df['Performance Type']==pt))]['Performance'].values
        for cat1 in perf_df['HF Cat'].unique():
            for pt1 in perf_df['Performance Type'].unique():
                p1 = perf_df.loc[((perf_df['HF Cat']==cat1)&(perf_df['Performance Type']==pt1))]['Performance'].values
                t,pval = st.ttest_ind(p,p1)
                
                if pval<0.05 and pt==pt1:
                    print(st.ttest_ind(p,p1))
                    print(cat, pt, cat1, pt1)
                    print()

Ttest_indResult(statistic=-3.5215446350820283, pvalue=0.024415994924790324)
people LF Performance landscapes LF Performance

Ttest_indResult(statistic=-4.452581505154506, pvalue=0.046918684356148724)
people LF Performance phones LF Performance

Ttest_indResult(statistic=-4.730108224493685, pvalue=0.017907979400470092)
people LF Performance buildings LF Performance

Ttest_indResult(statistic=-3.3404672812118674, pvalue=0.028823234371298818)
people LF Performance animals LF Performance

Ttest_indResult(statistic=-3.222453189097944, pvalue=0.032205381041511)
people LF Performance cars LF Performance

Ttest_indResult(statistic=-2.84084957523682, pvalue=0.036210323532722626)
landscapes HF Performance buildings HF Performance

Ttest_indResult(statistic=-2.911655949497841, pvalue=0.02692226493657512)
landscapes HF Performance animals HF Performance

Ttest_indResult(statistic=3.5215446350820283, pvalue=0.024415994924790324)
landscapes LF Performance people LF Performance

Ttest_indResult(stati

In [56]:
inits = []
conf_corr = []
counts = []
freq = []

for subj in df4['Inits'].unique():
    for f in ['h', 'l']:
        freq.extend([f.upper()]*6)
        c = df4.loc[((df4['Inits']==subj)&(df4['Freq']==f))]['Conf + Corr'].values
        for i in [-3,-2,-1,1,2,3]:
            conf_corr.append(i)
            counts.append(np.count_nonzero(c == i)/len(c))
        inits.extend([subj]*6)
    
conf_df = pd.DataFrame()
conf_df['Inits'] = inits
conf_df['Conf_corr'] = conf_corr
conf_df['Frac'] = counts
conf_df['Freq'] = freq

conf_df.head()

,Inits,Conf_corr,Frac,Freq
0,MM,-3,0.142857,H
1,MM,-2,0.047619,H
2,MM,-1,0.000000,H
3,MM,1,0.142857,H
4,MM,2,0.095238,H


In [57]:
f = iqplot.strip(data=conf_df, q='Frac', cats=['Conf_corr', 'Freq'], 
                 marker='dash',
#                  jitter=True, 
                 marker_kwargs=dict(alpha=0.5), 
                 title='Confidence for Low and High Frequency')
bokeh.plotting.show(f)

In [59]:
for cc in [-3,-2,-1,1,2,3]:
    h = conf_df.loc[conf_df['Conf_corr']==cc]
    h = h.loc[h['Freq']=='H']['Frac'].values
    l = conf_df.loc[conf_df['Conf_corr']==cc]
    l = l.loc[l['Freq']=='L']['Frac'].values

    print(cc)
    print(st.ttest_rel(h,l))
    print()

-3
Ttest_relResult(statistic=1.054147323015907, pvalue=0.304384647645212)

-2
Ttest_relResult(statistic=1.814717555434318, pvalue=0.08459612704459267)

-1
Ttest_relResult(statistic=1.528556773366893, pvalue=0.14203665292896894)

1
Ttest_relResult(statistic=1.9197252047018305, pvalue=0.0692716832945382)

2
Ttest_relResult(statistic=1.7951378836711114, pvalue=0.08775974713737812)

3
Ttest_relResult(statistic=-4.247251622417091, pvalue=0.00039462910163985253)



In [68]:
print(np.unique(df4.loc[df4['Freq']=='h']['Confidence'].values, return_counts=True)[1]/len(df4.loc[df4['Freq']=='h']['Confidence'].values))
print(np.unique(df4.loc[df4['Freq']=='l']['Confidence'].values, return_counts=True)[1]/len(df4.loc[df4['Freq']=='l']['Confidence'].values))

[0.15571776 0.26277372 0.58150852]
[0.08030303 0.17878788 0.74090909]
